In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing
import os
import sys
from scipy.ndimage import gaussian_filter1d
from scipy.signal import convolve
from brian2 import *

In [3]:
sys.path.append("../lib") # add code library to paths

# add lib to path 
from model_beta import Model

In [5]:
# TODO: turn this into  script
# os.mkdir('../output/')

In [6]:
netname = 'forward_nn'
model = Model(netname)  # initiate the model


../output/forward_nn


In [7]:
params = {
'sig_c' : 0.05  ,    # bipolar receptive fied size ~ 1/5 of the actual size [mm]
'tau1' : 0.04   ,    # excitation time constant of the temporal filter [s]
'tau2' : 0.0876 ,    # rebound time constant [s]
'bifw' : 0.     ,    # relative weight of rebound/excitation [1]

'scale_mV' : 20.,     # UPDATED 

'tauA' : 0.15   ,     # time constant of amacrine cells [s]
'tauB' : 0.08   ,     # time constant of bipolar cells [s]
'tauG' : 0.01   ,     # time contant of ganglion cells [s]

'wAB' : 10.     ,     # synaptic weight from bipolar to amacrine [Hz]
'wBA' : 0.    ,     # synaptic weight from amacrine to bipolar [Hz]
'wGB' : 4.0     ,     # synaptic weight from bipolar to gangion  [Hz].   #UPDATED
'wGA' : -2.0      ,     # synaptic weight from amacrine  to gangion  [Hz]

'slope' : 5     ,     # slope for ganglion cell recritifation [Hz/mV].   #UPDATED
'threshold' : 0 ,     # threshold for ganglion cell recritifation [Hz/mV]
'sig_pool' : 0.065 ,  # sigma for gaussian pooling in ganlion gells [mm]


'spacing' : 0.005  ,  # spacing of cells on the lattice [mm]
'dt' : 0.001      ,   # integration time step [s]
'N' : 512          ,  # number of neurons in each layer [1]


# idx = int(N/2)
# b =  0.160         # half bar width [mm]
# speed =  0.8       # speed [mm/s]  
}


In [8]:

# create connectivity matrix TODO write a class for this !!!

C = np.zeros((params['N'],params['N']))
ii = np.arange(0,params['N']-1,1).astype(int)  # list of source cells for rightward connections
ji = ii + 1                          # target cells of rightward connections


C[ii,ji] = 1

ii = np.arange(1,params['N'],1).astype(int)    # list of source cells for leftward connections
ji = ii - 1                         # target cells of leftward connections

C[ii,ji] = 1


In [9]:

model.set_params(params)
model.set_connectivity(C)


# simulate response to global step full contrast

In [10]:

dur = 3.                      # duration of the simulation [s]
tps = int(dur/params['dt'])             # number of time-points in the simulation [1]
time = np.arange(0,tps)*params['dt']    # time array for the simulation 1xtps, in [s]

start = int(1./params['dt'])
stop  = int(2./params['dt']) 

# initialize the stimulus
S = np.zeros((params['N'],tps))
S[:,start:stop] = 1


In [11]:
model.set_stimulus(S,'gStep')
model.simulate()

../output/forward_nn/gStep


# simulate response to global step  half contrast

In [12]:

dur = 3.                                # duration of the simulation [s]
tps = int(dur/params['dt'])             # number of time-points in the simulation [1]
time = np.arange(0,tps)*params['dt']    # time array for the simulation 1xtps, in [s]

start = int(1./params['dt'])
stop  = int(2./params['dt']) 

# initialize the stimulus
S = np.zeros((params['N'],tps))
S[:,start:stop] = .5


In [13]:
model.set_stimulus(S,'gStephalf')
model.simulate()

../output/forward_nn/gStephalf


In [ ]:
# loop over contrasts 
contrasts = np.arange(0.1,1.1,0.1)
S = np.zeros((params['N'],tps))

for c in contrasts: 
    S[:,start:stop] = c
    stimname = f'gStep_{c}'
    model.set_stimulus(S,stimname)
    model.simulate()

../output/forward_nn/gStep_0.1
../output/forward_nn/gStep_0.2
../output/forward_nn/gStep_0.30000000000000004
../output/forward_nn/gStep_0.4
../output/forward_nn/gStep_0.5
../output/forward_nn/gStep_0.6
../output/forward_nn/gStep_0.7000000000000001
../output/forward_nn/gStep_0.8
../output/forward_nn/gStep_0.9


In [ ]:
# loop over paramerter and save response, steady state
import pandas as pd
import joblib
from joblib import Parallel,delayed

joblib.cpu_count()

In [ ]:


wBAs = np.arange(1.,4.,0.2)
tauAs =np.round(np.logspace(0,1)*0.02,3)
wBAs = wBAs
tauAs = tauAs[::2]

print(len(wBAs),len(tauAs))
# wBAs = wBAs[:2]
# tauAs = tauAs[:2]

df = pd.DataFrame(columns = ['BC206', 'VB*','wGA','tauA'])
measure_point = int(2.9/params['dt'])



dur = 4.                                # duration of the simulation [s]
tps = int(dur/params['dt'])             # number of time-points in the simulation [1]
time = np.arange(0,tps)*params['dt']    # time array for the simulation 1xtps, in [s]

start = int(1./params['dt'])
stop  = int(3./params['dt']) 

# initialize the stimulus
S = np.zeros((params['N'],tps))
S[:,start:stop] = 1

model.set_stimulus(S,'gStep')
model.dt = 0.01

grid = np.array(np.meshgrid(wBAs,tauAs)).T.reshape(-1,2)

def run_one(par):


        wBA = par[0]
        tauA= par[1]
        params['tauA'] = tauA
        params['wGA'] = -1*wBA

        model.set_params(params)
        BC206 = model.simulate(save = False)
        VBstar = BC206[measure_point]
    

        dict = {'BC206': BC206,
                'wGA': wBA,
                'tauA': tauA,
                'VB*' : VBstar
                }
        
        return dict



X = Parallel(n_jobs = 40, verbose=10)(delayed(run_one)(g) for g in grid)


for xi in X:
        df.loc[len(df)] = xi

#save dataframe 
df.to_csv(f'{model.dir}/{model.stimname}/grid_steadystate.csv')


In [ ]:
plt.plot(df['BC206'].values[0])
plt.plot(df['BC206'].values[1])
plt.plot(df['BC206'].values[2])
plt.plot(df['BC206'].values[3])